# Problem Description

# Exploratory Data Analysis (EDA)

In [1]:
import pandas as pd
import numpy as np

In [2]:
movies = pd.read_csv('movies_dataset.csv')
movies.head()

,genre_ids,id,original_language,overview,popularity,release_date,title,vote_average,vote_count
0,"[878, 28, 12]",580489.0,en,After finding a host body in investigative rep...,6972.003,2021-09-30,Venom: Let There Be Carnage,7.2,4500.0
1,"[28, 35, 80, 53]",512195.0,en,An Interpol-issued Red Notice is a global aler...,4496.760,2021-11-04,Red Notice,6.8,2016.0
2,"[28, 12, 878, 14]",634649.0,en,Peter Parker is unmasked and no longer able to...,4062.446,2021-12-15,Spider-Man: No Way Home,8.4,84.0
3,"[16, 35, 10751]",585245.0,en,As Emily struggles to fit in at home and at sc...,2509.512,2021-11-10,Clifford the Big Red Dog,7.6,568.0
4,"[28, 12, 14]",566525.0,en,Shang-Chi must confront the past he thought he...,2453.799,2021-09-01,Shang-Chi and the Legend of the Ten Rings,7.8,4244.0


# Analysis (Model Building and Training)

# Results

# Conclusion